<a href="https://colab.research.google.com/github/YMGYM/python_deeplearning_pytorch/blob/main/Pytorch_part3~.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3-1 Convolutional Neural Network(CNN)

## CNN 기초

### Convolution Layer

이미지를 스캔하면서 이미지 Region Feature를 추출한다.

### Stride
Receptive Field 가 피쳐를 뽑을 때 이동하는 칸 수

### Padding
이미지 테두리 부분에도 Convolution을 적용하기 위해 사용

### Weight Sharing
슬라이드할 때마다 Window 의 같은 가중치를 공유..

### pooling
학습 속도 향상.
Layer 가중치를 반으로 줄여 줌

### Fully Connected Layer
Flatten 후 학습
